In [ ]:
#import shutil

#shutil.rmtree('/content/lightning_logs')
#shutil.rmtree('/content/logs')

**Deep Learning AA 2023/2024**
** **
Teachers:\
Fabrizio Silvestri (fsilvestri@diag.uniroma1.it)\
Antonio Purificato (antonio.purificato@uniroma1.it)\
Farooq Wani (wani@diag.uniroma1.it)\
Maria Sofia Bucarelli (bucarelli.diag@uniroma1.it) \


https://lightning-ai.github.io/tutorials/notebooks/lightning_examples/mnist-hello-world.html

In [1]:
import os

import pandas as pd
import seaborn as sn
import torch
import pytorch_lightning as pl
from IPython.core.display import display
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64

/tmp/ipykernel_151253/2700039455.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_151253/2700039455.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


### Reproducibility

To ensure full reproducibility from run to run you need to set seeds for pseudo-random generators, and set deterministic flag in Trainer.

In [2]:
pl.seed_everything(0)

Seed set to 0


0

## Class Dataset- Load the data

One very simple customized dataset

In [3]:
from sklearn.datasets import load_iris
data= load_iris()
print(data.DESCR[60:1210])
X = data.data
y = data.target
m, n = X.shape


**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Creator: R.A. Fisher
:Donor: Michael Marshall (MARSHALL%PLU@io

In [5]:
class IrisDataset(torch.utils.data.Dataset):
    # In this the class is very simple because we already data as two arrays
    def __init__(self):
        X, y = load_iris(return_X_y=True)
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
    def __getitem__(self, index):
        # Returns (xb, yb) pair
        x = self.X[index]
        y = self.y[index]
        return x, y
    def __len__(self):
        # Returns length
        return len(self.X)



iris_dataset = IrisDataset()
# we set the length of the training dataset, etc
length = len(iris_dataset)
train_len = int(length * 0.7)
val_len = int(length * 0.2 )
test_len = length - train_len - val_len

iris_train_dataset, iris_val_dataset, iris_test_dataset = \
torch.utils.data.random_split(iris_dataset, [train_len, val_len, test_len])

iris_train_loader = DataLoader(iris_train_dataset, batch_size=8, shuffle=True)
iris_val_loader = DataLoader(iris_val_dataset, batch_size=8)
iris_test_loader = DataLoader(iris_test_dataset, batch_size=8)

![testo del link](https://machinelearningmastery.com/wp-content/uploads/2019/02/Plot-of-a-Subset-of-Images-from-the-MNIST-Dataset.png)

One preload dataset

In [6]:
#dataloader
# Init DataLoader from MNIST Dataset
train_ds = MNIST(PATH_DATASETS, train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

100%|██████████| 9912422/9912422 [00:00<00:00, 51956130.46it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 71340220.16it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 18244557.40it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6962912.56it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



## Minimal example of boiler plate code with PyTorch

In [7]:
class MNIST_boiler(nn.Module):
  def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)

  def forward(self,x):
    return torch.relu(self.l1(x.view(x.size(0), -1)))


device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
model = MNIST_boiler().to(device)
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1)

In [10]:
# train
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:


epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306698  [    0/60000]
loss: 0.766460  [ 6400/60000]
loss: 1.022634  [12800/60000]
loss: 0.901100  [19200/60000]
loss: 1.022710  [25600/60000]
loss: 1.003174  [32000/60000]
loss: 0.891294  [38400/60000]
loss: 0.975485  [44800/60000]
loss: 1.084551  [51200/60000]
loss: 1.499024  [57600/60000]
Epoch 2
-------------------------------
loss: 0.832974  [    0/60000]
loss: 0.773510  [ 6400/60000]
loss: 0.979370  [12800/60000]
loss: 0.878403  [19200/60000]
loss: 1.001146  [25600/60000]
loss: 1.018126  [32000/60000]
loss: 0.888968  [38400/60000]
loss: 1.012458  [44800/60000]
loss: 1.077487  [51200/60000]
loss: 1.488508  [57600/60000]
Epoch 3
-------------------------------
loss: 0.835820  [    0/60000]
loss: 0.793407  [ 6400/60000]
loss: 0.946536  [12800/60000]
loss: 0.877589  [19200/60000]
loss: 1.000826  [25600/60000]
loss: 0.989130  [32000/60000]
loss: 0.879075  [38400/60000]
loss: 1.079047  [44800/60000]
loss: 1.099118  [51200/60000]
loss: 1.46

In [12]:
for batch, (X, y) in enumerate(train_loader):
    X, y = X.to(device), y.to(device)
    print(X.size(0))
    print(X.view(X.size(0), -1).shape)
    break

64
torch.Size([64, 784])


## Steps

1. pass pl.LightningModule instead of nn.Module to the module

2. move all required code under the relevant functions inside the module \\

3. you can remove .to(device) — Lightning moves the data coming from the LightningModule to devices automatically



In [13]:
class MNISTModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

# train model
model = MNISTModel()
trainer = pl.Trainer(max_epochs=5,
                     callbacks=[TQDMProgressBar(refresh_rate=20)],)
trainer.fit(model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/daniel/mambaforge/envs/ai/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder: /home/daniel/Git/Deep-Learning/notebooks/practice/practicum_1/lightning_logs

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031 

Epoch 4: 100%|██████████| 938/938 [00:05<00:00, 180.04it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 938/938 [00:05<00:00, 179.96it/s, v_num=0]


## More advanced stuff

### Managing the dataset
[Lighting DataModule](https://lightning.ai/docs/pytorch/stable/data/datamodule.html)
We'll see all the dataset specific pieces directly in the `LightningModule`.
This way, we can avoid writing extra code at the beginning of our script every time we want to run it.

---

### Note what the following built-in functions are doing:

1. [prepare_data()](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#prepare-data) 💾
    - This is where we can download the dataset. We point to our desired dataset and ask torchvision's `MNIST` dataset class to download if the dataset isn't found there.
    - **Note we do not make any state assignments in this function** (i.e. `self.something = ...`)

2. [setup(stage)](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#setup) ⚙️
    - Loads in data from file and prepares PyTorch tensor datasets for each split (train, val, test).
    - Setup expects a 'stage' arg which is used to separate logic for 'fit' and 'test'.
    - If you don't mind loading all your datasets at once, you can set up a condition to allow for both 'fit' related setup and 'test' related setup to run whenever `None` is passed to `stage` (or ignore it altogether and exclude any conditionals).
    - **Note this runs across all GPUs and it *is* safe to make state assignments here**

3. [x_dataloader()](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.core.hooks.DataHooks.html#pytorch_lightning.core.hooks.DataHooks.train_dataloader) ♻️
    - `train_dataloader()`, `val_dataloader()`, and `test_dataloader()` all return PyTorch `DataLoader` instances that are created by wrapping their respective datasets that we prepared in `setup()`

In [ ]:
from torch.utils.data import random_split, DataLoader

# Note - you must have torchvision installed for this example
from torchvision.datasets import MNIST
from torchvision import transforms


class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "./"):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

        if stage == "predict" or stage is None:
            self.mnist_predict = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=32)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=32)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=32)

    def predict_dataloader(self):
        return DataLoader(self.mnist_predict, batch_size=32)

### Validation and test

validation_step(self, batch, batch_idx)

test_step()




[LightModule](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html)

In [ ]:
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


    def test_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = F.mse_loss(x_hat, x)
        self.log("test_loss", test_loss)


    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = F.mse_loss(x_hat, x)
        self.log("val_loss", test_loss)




    def predict_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return F.softmax(x_hat)

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))

    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x):
        return self.l1(x)

In [ ]:
autoencoder = LitAutoEncoder(Encoder(), Decoder())

In [ ]:
dm = MNISTDataModule()
trainer = pl.Trainer(max_epochs=3,
                     callbacks=[TQDMProgressBar(refresh_rate=20)],)
trainer.fit(model=autoencoder, datamodule=dm)


In [ ]:
trainer.validate(datamodule=dm)
trainer.test(datamodule=dm)

## Callbacks 📞

A callback is a self-contained program that can be reused across projects. PyTorch Lightning comes with few built-in callbacks which are regularly used.
Learn more about callbacks in PyTorch Lightning https://pytorch-lightning.readthedocs.io/en/latest/extensions/callbacks.html


## Exercises



1.  Log other metrices like accuracy on Tensorboard. (Optional: use wandb https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch-lightning/Optimize_Pytorch_Lightning_models_with_Weights_%26_Biases.ipynb)  
2.  Complicate the Encoder-Decoder (e.g. add more layers)
3.  Play with more Callbacks -> ModelCheckpoint, Early Stopping.
4.  Follow this notebook by William Falcon, the creator of PyTorch Lightning https://colab.research.google.com/drive/1O_FI-QrCJcVDqUc-R5BKeckUF0AdYbyz?usp=sharing#scrollTo=TcMlRe7ywpe6 Don't worry if many things doesn't make sense now.

